In [3]:
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch
import pandas as pd

In [113]:
# architecture:

"""
Dense(12, input_dim=x_train.shape[1] -> 4),
ReLU(),
Dense(6),
ReLU(),
Dense(3),
Softmax()
"""

class Model(nn.Module):
  def __init__(self, in_features=4, h1=12, h2=6, out_features=3):
    super().__init__()
    self.fc1 = nn.Linear(in_features, h1) # first layer (input to 12)
    self.fc2 = nn.Linear(h1, h2) # second layer (12 to 6)
    self.out = nn.Linear(h2, out_features) # third layer (6 to 3 softmax); softmax is implicit

  def forward(self, x):
    # feed through the layers
    x = F.relu(self.fc1(x))
    x = F.relu(self.fc2(x))
    x = self.out(x)

    return x


In [114]:
import copy

class EarlyStopping:
    def __init__(self, model, save_best_weights=True, patience=5, min_delta=0.0, mode='min', min_epochs=5):
        if mode not in {"min", "max"}:
            raise ValueError(f"Invalid mode '{mode}'. Must be 'min' or 'max'.")
        if model is None:
            raise ValueError("The model must be provided.")
        if patience <= 0:
            raise ValueError("Patience must be at least 1.")

        self.model = model
        self.save_best_weights = save_best_weights
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.best_score = None
        self.counter = 0
        self.best_weights = None
        self.best_epoch = -1
        self.current_epoch = 0
        self.min_epochs = min_epochs

    def step(self, current_score):
      self.current_epoch += 1
      if self.best_score is None:
        improved = True
      elif self.mode == 'min':
        improved = current_score < self.best_score - self.min_delta
      else:  # mode == 'max'
        improved = current_score > self.best_score + self.min_delta

      if improved:
        self.best_score = current_score
        if self.save_best_weights:
          self.best_weights = copy.deepcopy(self.model.state_dict())
          self.best_epoch = self.current_epoch
        self.counter = 0
        return False # don't stop; improved
      else:
        self.counter += 1
        if self.counter >= self.patience:
          if self.save_best_weights and self.best_weights is not None:
            self.model.load_state_dict(self.best_weights)
          return self.current_epoch >= self.min_epochs # stop; counter exceeded patience - but only if we exceeded the minimum epoch count
        return False  # don't stop; we are being patient


In [115]:
df = pd.read_csv('Iris.csv')
df.drop('Id', inplace=True, axis=1)
df

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [116]:
x = df.drop('Species', axis=1)
y = pd.Categorical(df['Species']).codes

In [117]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled = scaler.fit_transform(x.values)
x = pd.DataFrame(scaled, columns=x.columns)

In [118]:
x = x.values

In [119]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)
x_train = torch.FloatTensor(x_train)
x_test = torch.FloatTensor(x_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [122]:
# Now that the data is all good to go, we can define the loss function (crossentropyloss due to multiclass classification), optimizer, lr and other hyperparameters
from sklearn.metrics import f1_score

torch.autograd.set_detect_anomaly(True)

model = Model()
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
early_stopping = EarlyStopping(model, save_best_weights=True, patience=4, min_delta=0.015, mode='min', min_epochs=10)

epochs = 200
losses = []

# actually train the model
for i in range(epochs):
  # forward pass
  model.train()
  y_pred = model(x_train)
  loss = criterion(y_pred, y_train)


  losses.append(loss.item())

  with torch.no_grad():
    pred_classes = torch.argmax(y_pred, dim=1).cpu().numpy()
    true_classes = y_train.cpu().numpy()
    f1 = f1_score(true_classes, pred_classes, average='weighted')

  print(f"[EPOCH {i}] - Loss: {loss} - F1: {f1}")

  # before we update the model, we need to see if we have to stop by early_stopping
  if (early_stopping.step(loss)):
    print(f"[EPOCH {i}] - STOPPED DUE TO EARLY STOPPING; BEST EPOCH WAS {early_stopping.best_epoch}")
    break

  optimizer.zero_grad() # reset the error gradient from the previous backpropagation
  loss.backward() # calculate the new error gradient through backpropagation
  optimizer.step() # update model params based on backpropagation gradient

model.eval()
# see the final accuracy on the test dataset
with torch.no_grad():
  y_test_pred = model.forward(x_test)
  pred_classes = torch.argmax(y_test_pred, dim=1).cpu().numpy()

  f1 = f1_score(y_test, pred_classes, average='weighted')
  true_classes = y_test.cpu().numpy()
  loss = criterion(y_test_pred, y_test)

print(f"\TEST PREDICTIONS: {pred_classes}, TRUE TEST VALUES: {y_test}")
print(f"FINAL LOSS: {loss} - FINAL F1: {f1}")


[EPOCH 0] - Loss: 1.174135684967041 - F1: 0.21638938403644287
[EPOCH 1] - Loss: 1.1709121465682983 - F1: 0.21638938403644287
[EPOCH 2] - Loss: 1.1677277088165283 - F1: 0.23055555555555554
[EPOCH 3] - Loss: 1.164570689201355 - F1: 0.27108274231678486
[EPOCH 4] - Loss: 1.1614409685134888 - F1: 0.2833962527964206


/usr/local/lib/python3.11/dist-packages/torch/autograd/graph.py:823: UserWarning: Error detected in AddmmBackward0. Traceback of forward call that caused the error:
  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>
    ColabKernelApp.launch_instance()
  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance
    app.start()
  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start
    self.io_loop.start()
  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever
    self._run_once()
  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once
    handle._run()
  File "/usr/lib/python3.1

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [6, 3]], which is output 0 of AsStridedBackward0, is at version 6; expected version 5 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!